<a href="https://colab.research.google.com/github/icehelix72/TSPredEML/blob/main/Rason_DataPreproc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install metpy pandas numpy certifi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.9/409.9 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 21.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import glob
import metpy.calc as mpcalc
from metpy.units import units
from datetime import datetime
import numpy as np
import requests
import certifi
import urllib.request
import time
import calendar
from bs4 import BeautifulSoup
from datetime import date, timedelta

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Specify the directory containing the Excel files
directory_path = "/content/drive/MyDrive/Skripsi/Data/Palu/Test"  # Replace with your directory path
file_pattern = "*.xlsx"  # Pattern to match Excel files

# Find all Excel files in the directory
excel_files = glob.glob(f"{directory_path}/{file_pattern}")

# Initialize a list to store the processed DataFrames for all sheets in all files
stability_indices = []
excel_files = sorted(excel_files)

In [ ]:
excel_files

['/content/drive/MyDrive/Skripsi/Data/Palu/Test/WAML012022.xlsx',
 '/content/drive/MyDrive/Skripsi/Data/Palu/Test/WAML012023.xlsx',
 '/content/drive/MyDrive/Skripsi/Data/Palu/Test/WAML022022.xlsx',
 '/content/drive/MyDrive/Skripsi/Data/Palu/Test/WAML022023.xlsx',
 '/content/drive/MyDrive/Skripsi/Data/Palu/Test/WAML032022.xlsx',
 '/content/drive/MyDrive/Skripsi/Data/Palu/Test/WAML032023.xlsx',
 '/content/drive/MyDrive/Skripsi/Data/Palu/Test/WAML042022.xlsx',
 '/content/drive/MyDrive/Skripsi/Data/Palu/Test/WAML042023.xlsx',
 '/content/drive/MyDrive/Skripsi/Data/Palu/Test/WAML052022.xlsx',
 '/content/drive/MyDrive/Skripsi/Data/Palu/Test/WAML052023.xlsx',
 '/content/drive/MyDrive/Skripsi/Data/Palu/Test/WAML062022.xlsx',
 '/content/drive/MyDrive/Skripsi/Data/Palu/Test/WAML062023.xlsx',
 '/content/drive/MyDrive/Skripsi/Data/Palu/Test/WAML072022.xlsx',
 '/content/drive/MyDrive/Skripsi/Data/Palu/Test/WAML072023.xlsx',
 '/content/drive/MyDrive/Skripsi/Data/Palu/Test/WAML082022.xlsx',
 '/content

In [ ]:
file = 'WAML122023'
file_name = '/content/drive/MyDrive/Skripsi/Data/Palu/Test/'+file+'.xlsx'
print(f"Processing file: {file_name}")
sheets_dict = pd.read_excel(file_name, engine="openpyxl", sheet_name=None)
for sheet_name, df in sheets_dict.items():
  print(f"Processing sheet: {sheet_name} from file: {file_name}")
  if {'pressure', 'temperature', 'dewpoint', 'speed', 'direction', 'height'}.issubset(df.columns):
    df = df.dropna(subset=('temperature', 'dewpoint', 'direction', 'speed'), how='all').reset_index(drop=True)
    try:
      # Extract data and convert to appropriate units
      pressure = df['pressure'].values * units.hPa
      temperature = df['temperature'].values * units.degC
      dewpoint = df['dewpoint'].values * units.degC
      speed = df['speed'].values * units.knots
      direction = df['direction'].values * units.degrees
      height = df['height'].values * units.meter
      relative_humidity = mpcalc.relative_humidity_from_dewpoint(temperature, dewpoint)
      mixing_ratio = mpcalc.mixing_ratio_from_relative_humidity(pressure, temperature, relative_humidity)

      # Gálvez-Davison Index
      gdi = mpcalc.galvez_davison_index(pressure, temperature, mixing_ratio, pressure[0])

      # Wind and dewpoint parameter
      pressure1 = 200 * units.hPa
      wind_200 = speed[pressure == pressure1]
      pressure2 = 500 * units.hPa
      wind_500 = speed[pressure == pressure2]
      pressure3 = 850 * units.hPa
      wind_850 = speed[pressure == pressure3]
      dewpoint_850 = dewpoint[pressure == pressure3]
      dewpoint_sfc = dewpoint[0]

      stability_indices.append({
          'time': datetime.strptime(sheet_name, '%Y-%m-%d %HZ'),
          'GDI': gdi.magnitude,
          '200 mb Wind': wind_200.magnitude[0],
          '500 mb Wind': wind_500.magnitude[0],
          '850 mb Wind': wind_850.magnitude[0],
          '850 mb Dewpoint': dewpoint_850.magnitude[0],
          'Surface Dewpoint': dewpoint_sfc.magnitude
          })
    except ValueError as e:
      print(f"Error processing {file_name}, sheet {sheet_name}: {e}")
    except IndexError as e:
      print(f"Error processing {file_name}, sheet {sheet_name}: {e}")

stability_indices_df = pd.DataFrame(stability_indices)
stability_indices_df.to_excel('/content/drive/MyDrive/Skripsi/Data/Palu/Test/Output/'+file+'.xlsx', index=False)

Processing file: /content/drive/MyDrive/Skripsi/Data/Palu/Test/WAML122023.xlsx


KeyboardInterrupt: 

In [ ]:
for file_name in excel_files:
  print(f"Processing file: {file_name}")
  # Reading multiple sheets from an Excel file
  sheets_dict = pd.read_excel(file_name, engine="openpyxl", sheet_name=None)
  # Accessing individual sheets and displaying their contents
  for sheet_name, df in sheets_dict.items():
    print(f"Processing sheet: {sheet_name} from file: {file_name}")
    if {'pressure', 'temperature', 'dewpoint', 'speed', 'direction', 'height'}.issubset(df.columns):
      df = df.dropna(subset=('temperature', 'dewpoint', 'direction', 'speed'), how='all').reset_index(drop=True)
      try:
        # Extract data and convert to appropriate units
        pressure = df['pressure'].values * units.hPa
        temperature = df['temperature'].values * units.degC
        dewpoint = df['dewpoint'].values * units.degC
        speed = df['speed'].values * units.knots
        direction = df['direction'].values * units.degrees
        height = df['height'].values * units.meter
        relative_humidity = mpcalc.relative_humidity_from_dewpoint(temperature, dewpoint)
        mixing_ratio = mpcalc.mixing_ratio_from_relative_humidity(pressure, temperature, relative_humidity)

        # Gálvez-Davison Index
        gdi = mpcalc.galvez_davison_index(pressure, temperature, mixing_ratio, pressure[0])

        # Wind and dewpoint parameter
        pressure1 = 200 * units.hPa
        wind_200 = speed[pressure == pressure1]
        pressure2 = 500 * units.hPa
        wind_500 = speed[pressure == pressure2]
        pressure3 = 850 * units.hPa
        wind_850 = speed[pressure == pressure3]
        dewpoint_850 = dewpoint[pressure == pressure3]
        dewpoint_sfc = dewpoint[0]

        stability_indices.append({
            'sheet': datetime.strptime(sheet_name, '%Y-%m-%d %HZ'),
            'GDI': gdi.magnitude,
            '200 mb Wind': wind_200.magnitude[0],
            '500 mb Wind': wind_500.magnitude[0],
            '850 mb Wind': wind_850.magnitude[0],
            '850 mb Dewpoint': dewpoint_850.magnitude[0],
            'Surface Dewpoint': dewpoint_sfc.magnitude
            })
      except ValueError as e:
        print(f"Error processing {file_name}, sheet {sheet_name}: {e}")
      except IndexError as e:
        print(f"Error processing {file_name}, sheet {sheet_name}: {e}")

Processing file: /content/drive/MyDrive/Skripsi/Data/Palu/Test/WAML012022.xlsx
Processing sheet: 2022-01-01 00Z from file: /content/drive/MyDrive/Skripsi/Data/Palu/Test/WAML012022.xlsx
Processing sheet: 2022-01-01 12Z from file: /content/drive/MyDrive/Skripsi/Data/Palu/Test/WAML012022.xlsx
Processing sheet: 2022-01-02 00Z from file: /content/drive/MyDrive/Skripsi/Data/Palu/Test/WAML012022.xlsx
Processing sheet: 2022-01-02 12Z from file: /content/drive/MyDrive/Skripsi/Data/Palu/Test/WAML012022.xlsx
Processing sheet: 2022-01-03 00Z from file: /content/drive/MyDrive/Skripsi/Data/Palu/Test/WAML012022.xlsx
Processing sheet: 2022-01-03 12Z from file: /content/drive/MyDrive/Skripsi/Data/Palu/Test/WAML012022.xlsx
Processing sheet: 2022-01-04 00Z from file: /content/drive/MyDrive/Skripsi/Data/Palu/Test/WAML012022.xlsx
Processing sheet: 2022-01-04 12Z from file: /content/drive/MyDrive/Skripsi/Data/Palu/Test/WAML012022.xlsx
Processing sheet: 2022-01-05 00Z from file: /content/drive/MyDrive/Skrips

<ipython-input-11-c85c184c7c80>:22: UserWarning: Interpolation point out of data bounds encountered
  gdi = mpcalc.galvez_davison_index(pressure, temperature, mixing_ratio, pressure[0])
<ipython-input-11-c85c184c7c80>:22: UserWarning: Interpolation point out of data bounds encountered
  gdi = mpcalc.galvez_davison_index(pressure, temperature, mixing_ratio, pressure[0])


Processing sheet: 2022-02-11 00Z from file: /content/drive/MyDrive/Skripsi/Data/Palu/Test/WAML022022.xlsx
Processing sheet: 2022-02-11 12Z from file: /content/drive/MyDrive/Skripsi/Data/Palu/Test/WAML022022.xlsx
Processing sheet: 2022-02-12 00Z from file: /content/drive/MyDrive/Skripsi/Data/Palu/Test/WAML022022.xlsx
Processing sheet: 2022-02-12 12Z from file: /content/drive/MyDrive/Skripsi/Data/Palu/Test/WAML022022.xlsx
Processing sheet: 2022-02-13 00Z from file: /content/drive/MyDrive/Skripsi/Data/Palu/Test/WAML022022.xlsx
Processing sheet: 2022-02-13 12Z from file: /content/drive/MyDrive/Skripsi/Data/Palu/Test/WAML022022.xlsx
Processing sheet: 2022-02-14 00Z from file: /content/drive/MyDrive/Skripsi/Data/Palu/Test/WAML022022.xlsx
Processing sheet: 2022-02-14 12Z from file: /content/drive/MyDrive/Skripsi/Data/Palu/Test/WAML022022.xlsx
Processing sheet: 2022-02-15 00Z from file: /content/drive/MyDrive/Skripsi/Data/Palu/Test/WAML022022.xlsx
Processing sheet: 2022-02-15 12Z from file: /c

<ipython-input-11-c85c184c7c80>:22: UserWarning: Interpolation point out of data bounds encountered
  gdi = mpcalc.galvez_davison_index(pressure, temperature, mixing_ratio, pressure[0])


Error processing /content/drive/MyDrive/Skripsi/Data/Palu/Test/WAML042023.xlsx, sheet 2023-04-20 12Z: index 0 is out of bounds for axis 0 with size 0
Processing sheet: 2023-04-21 00Z from file: /content/drive/MyDrive/Skripsi/Data/Palu/Test/WAML042023.xlsx
Processing sheet: 2023-04-21 12Z from file: /content/drive/MyDrive/Skripsi/Data/Palu/Test/WAML042023.xlsx
Processing sheet: 2023-04-22 00Z from file: /content/drive/MyDrive/Skripsi/Data/Palu/Test/WAML042023.xlsx
Processing sheet: 2023-04-22 12Z from file: /content/drive/MyDrive/Skripsi/Data/Palu/Test/WAML042023.xlsx
Processing sheet: 2023-04-23 00Z from file: /content/drive/MyDrive/Skripsi/Data/Palu/Test/WAML042023.xlsx
Processing sheet: 2023-04-23 12Z from file: /content/drive/MyDrive/Skripsi/Data/Palu/Test/WAML042023.xlsx
Processing sheet: 2023-04-24 00Z from file: /content/drive/MyDrive/Skripsi/Data/Palu/Test/WAML042023.xlsx
Processing sheet: 2023-04-24 12Z from file: /content/drive/MyDrive/Skripsi/Data/Palu/Test/WAML042023.xlsx
Pr

<ipython-input-11-c85c184c7c80>:22: UserWarning: Interpolation point out of data bounds encountered
  gdi = mpcalc.galvez_davison_index(pressure, temperature, mixing_ratio, pressure[0])


Processing sheet: 2023-07-31 12Z from file: /content/drive/MyDrive/Skripsi/Data/Palu/Test/WAML072023.xlsx
Processing file: /content/drive/MyDrive/Skripsi/Data/Palu/Test/WAML082022.xlsx
Processing sheet: 2022-08-01 00Z from file: /content/drive/MyDrive/Skripsi/Data/Palu/Test/WAML082022.xlsx
Processing sheet: 2022-08-01 12Z from file: /content/drive/MyDrive/Skripsi/Data/Palu/Test/WAML082022.xlsx
Processing sheet: 2022-08-02 00Z from file: /content/drive/MyDrive/Skripsi/Data/Palu/Test/WAML082022.xlsx
Processing sheet: 2022-08-02 12Z from file: /content/drive/MyDrive/Skripsi/Data/Palu/Test/WAML082022.xlsx
Processing sheet: 2022-08-03 00Z from file: /content/drive/MyDrive/Skripsi/Data/Palu/Test/WAML082022.xlsx
Processing sheet: 2022-08-03 12Z from file: /content/drive/MyDrive/Skripsi/Data/Palu/Test/WAML082022.xlsx
Processing sheet: 2022-08-04 00Z from file: /content/drive/MyDrive/Skripsi/Data/Palu/Test/WAML082022.xlsx
Processing sheet: 2022-08-05 00Z from file: /content/drive/MyDrive/Skrips

<ipython-input-11-c85c184c7c80>:22: UserWarning: Interpolation point out of data bounds encountered
  gdi = mpcalc.galvez_davison_index(pressure, temperature, mixing_ratio, pressure[0])


Processing sheet: 2023-12-12 12Z from file: /content/drive/MyDrive/Skripsi/Data/Palu/Test/WAML122023.xlsx
Processing sheet: 2023-12-13 00Z from file: /content/drive/MyDrive/Skripsi/Data/Palu/Test/WAML122023.xlsx
Processing sheet: 2023-12-13 12Z from file: /content/drive/MyDrive/Skripsi/Data/Palu/Test/WAML122023.xlsx
Error processing /content/drive/MyDrive/Skripsi/Data/Palu/Test/WAML122023.xlsx, sheet 2023-12-13 12Z: index 0 is out of bounds for axis 0 with size 0
Processing sheet: 2023-12-14 00Z from file: /content/drive/MyDrive/Skripsi/Data/Palu/Test/WAML122023.xlsx
Processing sheet: 2023-12-14 12Z from file: /content/drive/MyDrive/Skripsi/Data/Palu/Test/WAML122023.xlsx
Processing sheet: 2023-12-15 00Z from file: /content/drive/MyDrive/Skripsi/Data/Palu/Test/WAML122023.xlsx
Error processing /content/drive/MyDrive/Skripsi/Data/Palu/Test/WAML122023.xlsx, sheet 2023-12-15 00Z: index 0 is out of bounds for axis 0 with size 0
Processing sheet: 2023-12-15 12Z from file: /content/drive/MyDri

In [ ]:
stability_indices_df = pd.DataFrame(stability_indices)
stability_indices_df.to_excel('/content/drive/MyDrive/Skripsi/Data/Palu/Test/stability_indices.xlsx', index=False)

In [ ]:
for file_name in excel_files:
  print(f"Processing file: {file_name}")
  # Reading multiple sheets from an Excel file
  sheets_dict = pd.read_excel(file_name, engine="openpyxl", sheet_name=None)
  # Accessing individual sheets and displaying their contents
  for sheet_name, df in sheets_dict.items():
    print(f"Processing sheet: {sheet_name} from file: {file_name}")
    if {'pressure', 'temperature', 'dewpoint', 'speed', 'direction', 'height'}.issubset(df.columns):
      df = df.dropna(subset=('temperature', 'dewpoint', 'direction', 'speed'), how='all').reset_index(drop=True)
      # Extract data and convert to appropriate units
      pressure = df['pressure'].values * units.hPa
      temperature = df['temperature'].values * units.degC
      dewpoint = df['dewpoint'].values * units.degC
      speed = df['speed'].values * units.knots
      direction = df['direction'].values * units.degrees
      height = df['height'].values * units.meter
      relative_humidity = mpcalc.relative_humidity_from_dewpoint(temperature, dewpoint)
      mixing_ratio = mpcalc.mixing_ratio_from_relative_humidity(pressure, temperature, relative_humidity)

      # Gálvez-Davison Index
      gdi = mpcalc.galvez_davison_index(pressure, temperature, mixing_ratio, pressure[0])

      # Wind and dewpoint parameter
      pressure1 = 200 * units.hPa
      wind_200 = speed[pressure == pressure1]
      pressure2 = 500 * units.hPa
      wind_500 = speed[pressure == pressure2]
      pressure3 = 850 * units.hPa
      wind_850 = speed[pressure == pressure3]
      dewpoint_850 = dewpoint[pressure == pressure3]
      dewpoint_sfc = dewpoint[0]

      stability_indices.append({
          'sheet': datetime.strptime(sheet_name, '%Y-%m-%d %HZ'),
          'GDI': gdi.magnitude,
          '200 mb Wind': wind_200.magnitude,
          '500 mb Wind': wind_500.magnitude,
          '850 mb Wind': wind_850.magnitude,
          '850 mb Dewpoint': dewpoint_850.magnitude,
          'Surface Dewpoint': dewpoint_sfc.magnitude
          })

Processing file: /content/drive/MyDrive/Skripsi/Data/Palu/Test/WAML012022.xlsx
Processing sheet: 2022-01-01 00Z from file: /content/drive/MyDrive/Skripsi/Data/Palu/Test/WAML012022.xlsx
Processing sheet: 2022-01-01 12Z from file: /content/drive/MyDrive/Skripsi/Data/Palu/Test/WAML012022.xlsx
Processing sheet: 2022-01-02 00Z from file: /content/drive/MyDrive/Skripsi/Data/Palu/Test/WAML012022.xlsx
Processing sheet: 2022-01-02 12Z from file: /content/drive/MyDrive/Skripsi/Data/Palu/Test/WAML012022.xlsx
Processing sheet: 2022-01-03 00Z from file: /content/drive/MyDrive/Skripsi/Data/Palu/Test/WAML012022.xlsx
Processing sheet: 2022-01-03 12Z from file: /content/drive/MyDrive/Skripsi/Data/Palu/Test/WAML012022.xlsx
Processing sheet: 2022-01-04 00Z from file: /content/drive/MyDrive/Skripsi/Data/Palu/Test/WAML012022.xlsx
Processing sheet: 2022-01-04 12Z from file: /content/drive/MyDrive/Skripsi/Data/Palu/Test/WAML012022.xlsx
Processing sheet: 2022-01-05 00Z from file: /content/drive/MyDrive/Skrips

<ipython-input-19-ea309a266190>:21: UserWarning: Interpolation point out of data bounds encountered
  gdi = mpcalc.galvez_davison_index(pressure, temperature, mixing_ratio, pressure[0])


Processing sheet: 2022-02-15 12Z from file: /content/drive/MyDrive/Skripsi/Data/Palu/Test/WAML022022.xlsx
Processing sheet: 2022-02-16 00Z from file: /content/drive/MyDrive/Skripsi/Data/Palu/Test/WAML022022.xlsx
Processing sheet: 2022-02-16 12Z from file: /content/drive/MyDrive/Skripsi/Data/Palu/Test/WAML022022.xlsx
Processing sheet: 2022-02-17 00Z from file: /content/drive/MyDrive/Skripsi/Data/Palu/Test/WAML022022.xlsx
Processing sheet: 2022-02-17 12Z from file: /content/drive/MyDrive/Skripsi/Data/Palu/Test/WAML022022.xlsx
Processing sheet: 2022-02-18 00Z from file: /content/drive/MyDrive/Skripsi/Data/Palu/Test/WAML022022.xlsx
Processing sheet: 2022-02-18 12Z from file: /content/drive/MyDrive/Skripsi/Data/Palu/Test/WAML022022.xlsx
Processing sheet: 2022-02-19 00Z from file: /content/drive/MyDrive/Skripsi/Data/Palu/Test/WAML022022.xlsx
Processing sheet: 2022-02-19 12Z from file: /content/drive/MyDrive/Skripsi/Data/Palu/Test/WAML022022.xlsx
Processing sheet: 2022-02-20 00Z from file: /c

<ipython-input-19-ea309a266190>:21: UserWarning: Interpolation point out of data bounds encountered
  gdi = mpcalc.galvez_davison_index(pressure, temperature, mixing_ratio, pressure[0])



Processing sheet: 2022-02-22 12Z from file: /content/drive/MyDrive/Skripsi/Data/Palu/Test/WAML022022.xlsx
Processing sheet: 2022-02-23 00Z from file: /content/drive/MyDrive/Skripsi/Data/Palu/Test/WAML022022.xlsx
Processing sheet: 2022-02-23 12Z from file: /content/drive/MyDrive/Skripsi/Data/Palu/Test/WAML022022.xlsx
Processing sheet: 2022-02-24 00Z from file: /content/drive/MyDrive/Skripsi/Data/Palu/Test/WAML022022.xlsx
Processing sheet: 2022-02-24 12Z from file: /content/drive/MyDrive/Skripsi/Data/Palu/Test/WAML022022.xlsx
Processing sheet: 2022-02-25 00Z from file: /content/drive/MyDrive/Skripsi/Data/Palu/Test/WAML022022.xlsx
Processing sheet: 2022-02-25 12Z from file: /content/drive/MyDrive/Skripsi/Data/Palu/Test/WAML022022.xlsx
Processing sheet: 2022-02-26 00Z from file: /content/drive/MyDrive/Skripsi/Data/Palu/Test/WAML022022.xlsx
Processing sheet: 2022-02-26 12Z from file: /content/drive/MyDrive/Skripsi/Data/Palu/Test/WAML022022.xlsx
Processing sheet: 2022-02-27 00Z from file: /

<ipython-input-19-ea309a266190>:21: UserWarning: Interpolation point out of data bounds encountered
  gdi = mpcalc.galvez_davison_index(pressure, temperature, mixing_ratio, pressure[0])


Processing sheet: 2023-04-21 12Z from file: /content/drive/MyDrive/Skripsi/Data/Palu/Test/WAML042023.xlsx
Processing sheet: 2023-04-22 00Z from file: /content/drive/MyDrive/Skripsi/Data/Palu/Test/WAML042023.xlsx
Processing sheet: 2023-04-22 12Z from file: /content/drive/MyDrive/Skripsi/Data/Palu/Test/WAML042023.xlsx
Processing sheet: 2023-04-23 00Z from file: /content/drive/MyDrive/Skripsi/Data/Palu/Test/WAML042023.xlsx
Processing sheet: 2023-04-23 12Z from file: /content/drive/MyDrive/Skripsi/Data/Palu/Test/WAML042023.xlsx
Processing sheet: 2023-04-24 00Z from file: /content/drive/MyDrive/Skripsi/Data/Palu/Test/WAML042023.xlsx
Processing sheet: 2023-04-24 12Z from file: /content/drive/MyDrive/Skripsi/Data/Palu/Test/WAML042023.xlsx
Processing sheet: 2023-04-25 00Z from file: /content/drive/MyDrive/Skripsi/Data/Palu/Test/WAML042023.xlsx
Processing sheet: 2023-04-25 12Z from file: /content/drive/MyDrive/Skripsi/Data/Palu/Test/WAML042023.xlsx
Processing sheet: 2023-04-26 00Z from file: /c

ValueError: Data not provided for 950hPa or higher pressure. GDI requires 950hPa temperature and dewpoint data, see referenced paper section 3.d. in docstring for discussion of extrapolating sounding data below terrain surface in high-elevation regions.
Indices without a 950hPa or higher datapoint
Max provided pressures:
101.0 hectopascal

In [ ]:
file_name = '/content/drive/MyDrive/Skripsi/Data/Palu/Test/WAML012022.xlsx'
sheet_name = '2022-01-01 00Z'

df = pd.read_excel(file_name, engine="openpyxl", sheet_name=sheet_name)
df = df.dropna(subset=('temperature', 'dewpoint', 'direction', 'speed'), how='any').reset_index(drop=True)
pressure = df['pressure'].values * units.hPa
temperature = df['temperature'].values * units.degC
dewpoint = df['dewpoint'].values * units.degC
height = df['height'].values * units.meter
speed = df['speed'].values * units.knots
direction = df['direction'].values * units.degrees
relative_humidity = mpcalc.relative_humidity_from_dewpoint(temperature, dewpoint)
mixing_ratio = mpcalc.mixing_ratio_from_relative_humidity(pressure, temperature, relative_humidity)

# Gálvez-Davison Index
gdi = mpcalc.galvez_davison_index(pressure, temperature, mixing_ratio, pressure[0])

# Wind and dewpoint parameter
pressure1 = 200 * units.hPa
wind_200 = speed[pressure == pressure1]
pressure2 = 500 * units.hPa
wind_500 = speed[pressure == pressure2]
pressure3 = 850 * units.hPa
wind_850 = speed[pressure == pressure3]
dewpoint_850 = dewpoint[pressure == pressure3]
dewpoint_sfc = dewpoint[0]

dewpoint_sfc

<Quantity(22.6, 'degree_Celsius')>

In [ ]:
for year in year:
    for month in months:
      for day in days:
        for hour in hours:
          print("Processing ", year, month, day, hour)
          url = 'https://weather.uwyo.edu/cgi-bin/sounding?region=seasia&TYPE=TEXT%3ALIST&YEAR='+str(year)+'&MONTH='+str(month).zfill(2)+'&FROM='+str(day).zfill(2)+str(hour).zfill(2)+'&TO='+str(day).zfill(2)+str(hour).zfill(2)+'&STNM=97072'
          response = requests.get(url, verify=False) # Disable SSL verification
          response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)
          soup = BeautifulSoup(response.text, "html.parser")

          for i in range(1, len(soup.findAll('pre'))+1):
              if (i % 2 == 1):
                data00 = soup.findAll('pre')[i].text # 00
                dict00 = {}
                dict12 = {}

                if len(soup.findAll('pre')) > 0:
                  data00 = soup.findAll('pre')[i].text # 00

                  for isi in data00.split("\n"):
                    if len(isi) >= 2:
                      dict00[isi.split(":")[0].strip()] = isi.split(":")[1]
                      list00.append([
                          dict00['Station number'] if 'Station number' in dict00 else None,
                          dict00['Observation time'] if 'Observation time' in dict00 else None,
                          dict00['LIFT computed using virtual temperature'] if 'LIFT computed using virtual temperature' in dict00 else None,
                          dict00['CAPE using virtual temperature'] if 'CAPE using virtual temperature' in dict00 else None,
                          dict00['CINS using virtual temperature'] if 'CINS using virtual temperature' in dict00 else None,
                          dict00['K index'] if 'K index' in dict00 else None,
                          dict00['Totals totals index'] if 'Totals totals index' in dict00 else None,
                          dict00['Convective Available Potential Energy'] if 'Convective Available Potential Energy' in dict00 else None,
                          dict00['Precipitable water [mm] for entire sounding'] if 'Precipitable water [mm] for entire sounding' in dict00 else None])
                      df00 = pd.DataFrame(list00, columns =['ID', 'Time','SI','LI','SWEAT','KI','TTI','CAPE','PW'])
                      df00.to_csv('96011_indeks_labilitas.csv', sep=',', encoding='utf-8', index=False)

In [ ]:
from datetime import datetime, timedelta

def generate_dates(start_date, end_date, interval_hours):
    """
    Generates lists of year, month, day, and hour between start_date and end_date,
    inclusive, with a specified interval in hours.

    Args:
        start_date (datetime): The starting datetime.
        end_date (datetime): The ending datetime.
        interval_hours (int): The interval between datetimes in hours.

    Returns:
        tuple: A tuple containing four lists: years, months, days, hours.
    """
    current_date = start_date
    years = []
    months = []
    days = []
    hours = []

    while current_date <= end_date:
        years.append(current_date.year)
        months.append(current_date.month)
        days.append(current_date.day)
        hours.append(current_date.hour)
        current_date += timedelta(hours=interval_hours)

    return years, months, days, hours

# Example usage:
start_date = datetime(2022, 1, 1, 0, 0, 0)
end_date = datetime(2023, 12, 31, 23, 59, 59)
interval_hours = 12

years, months, days, hours = generate_dates(start_date, end_date, interval_hours)

#To see the output, run the code.
#print(years)
#print(months)
#print(days)
#print(hours)

In [ ]:
# Skrip fix
for file_name in excel_files:
  print(f"Processing file: {file_name}")

  # Reading multiple sheets from an Excel file
  sheets_dict = pd.read_excel(file_name, engine="openpyxl", sheet_name=None)

    # Accessing individual sheets and displaying their contents
  for sheet_name, df in sheets_dict.items():
    print(f"Processing sheet: {sheet_name} from file: {file_name}")
    if {'pressure', 'temperature', 'dewpoint', 'speed', 'direction', 'height'}.issubset(df.columns):
      df = df.dropna(subset=('temperature', 'dewpoint', 'direction', 'speed'), how='all').reset_index(drop=True)
      try:
        # Extract data and convert to appropriate units
        pressure = df['pressure'].values * units.hPa
        temperature = df['temperature'].values * units.degC
        dewpoint = df['dewpoint'].values * units.degC
        speed = df['speed'].values * units.knots
        direction = df['direction'].values * units.degrees
        height = df['height'].values * units.meter
        relative_humidity = mpcalc.relative_humidity_from_dewpoint(temperature, dewpoint)
        mixing_ratio = mpcalc.mixing_ratio_from_relative_humidity(pressure, temperature, relative_humidity)

        # Calculate parcel profile
        parcel_prof = mpcalc.parcel_profile(pressure, temperature[0], dewpoint[0])

        # LCL, LFC, CCL and EL
        lcl_pressure, lcl_temperature = mpcalc.lcl(pressure[0], temperature[0], dewpoint[0])
        lfc_pressure, lfc_temperature = mpcalc.lfc(pressure, temperature, dewpoint)
        el_pressure, el_temperature = mpcalc.el(pressure, temperature, dewpoint)


        # Mean-layer parcel characteristics (50-hPa depth)
        #ml_t, ml_td = mpcalc.mixed_layer(pressure, temperature, dewpoint)
        #ml_p, _, _ = mpcalc.mixed_parcel(pressure, temperature, dewpoint)
        #mlcape, mlcin = mpcalc.mixed_layer_cape_cin(pressure, temperature, dewpoint)

        # Most unstable parcel characteristics (50-hPa depth)
        #mu_p, mu_t, mu_td, _ = mpcalc.most_unstable_parcel(pressure, temperature, dewpoint)
        #mucape, mucin = mpcalc.most_unstable_cape_cin(pressure, temperature, dewpoint)

        # Surface-based CAPE and CIN
        sb_cape, sb_cin = mpcalc.surface_based_cape_cin(pressure, temperature, dewpoint)

        # Stability indices
        parcel_p, parcel_t, parcel_td = mpcalc.mixed_parcel(pressure, temperature, dewpoint, height=height, depth=500 * units.meter)
        above = height > 500 * units.m
        press = np.concatenate([[parcel_p], pressure[above]])
        temp = np.concatenate([[parcel_t], temperature[above]])
        dewpt = np.concatenate([[parcel_td], dewpoint[above]])
        mixed_prof = mpcalc.parcel_profile(press, parcel_t, parcel_td)
        lift_index = mpcalc.lifted_index(press, temp, mixed_prof)
        # CAPE and CIN
        p, t, d, prof = mpcalc.parcel_profile_with_lcl(press, temp, dewpt)
        cape, cin = mpcalc.cape_cin(p, t, d, prof)

        # Another Stability Indices
        k = mpcalc.k_index(pressure, temperature, dewpoint)
        tti = mpcalc.total_totals_index(pressure, temperature, dewpoint)
        cti = mpcalc.cross_totals(pressure, temperature, dewpoint)
        vti = mpcalc.vertical_totals(pressure, temperature)
        si = mpcalc.showalter_index(pressure, temperature, dewpoint)
        sweat = mpcalc.sweat_index(pressure, temperature, dewpoint, speed, direction)
        gdi = mpcalc.galvez_davison_index(pressure, temperature, mixing_ratio, pressure[0])

        # Storm motion and critical angle
        u, v = mpcalc.wind_components(speed, direction)
        (u_storm, v_storm), *_ = mpcalc.bunkers_storm_motion(pressure, u, v, height)
        critical_angle = mpcalc.critical_angle(pressure, u, v, height, u_storm, v_storm)

        # Compute SRH, given a motion vector toward the NE at 9.9 m/s
        *_, total_helicity = mpcalc.storm_relative_helicity(height, u, v, depth=1 * units.km, storm_u=u_storm, storm_v=v_storm)

        # Append results to the list
        stability_indices.append({
            'sheet': datetime.strptime(sheet_name, '%Y-%m-%d %HZ'),
            'LCL_Pressure': lcl_pressure.magnitude,
            'LFC_Pressure': lfc_pressure.magnitude,
            'EL_Pressure': el_pressure.magnitude,
            'CAPE': cape.magnitude,
            'CIN': cin.magnitude,
            'lifted_index': lift_index.magnitude[0],
            'K_Index': k.magnitude,
            'Total_Totals_Index': tti.magnitude,
            'Cross_Totals_Index': cti.magnitude,
            'Vertical_Totals_Index': vti.magnitude,
            'Showalter_Index': si.magnitude[0],
            'SWEAT_Index': sweat.magnitude[0],
            'Galvez_Davis_Index': gdi.magnitude,
            #'Mean_Layer_Temperature': ml_t.magnitude,
            #'Mean_Layer_Dewpoint': ml_td.magnitude,
            #'Mean_Layer_Pressure': ml_p.magnitude,
            #'ML_CAPE': mlcape.magnitude,
            #'ML_CIN': mlcin.magnitude,
            #'Most_Unstable_Temperature': mu_t.magnitude,
            #'Most_Unstable_Dewpoint': mu_td.magnitude,
            #'Most_Unstable_Pressure': mu_p.magnitude,
            #'MU_CAPE': mucape.magnitude,
            #'MU_CIN': mucin.magnitude,
            'SB_CAPE': sb_cape.magnitude,
            'SB_CIN': sb_cin.magnitude,
            #'Mean_Parcel_Pressure': mean_parcel_pressure.magnitude,
            #'MU_Parcel_Pressure': mu_parcel_pressure.magnitude,
            #'Bunkers_Storm_Motion': storm_motion,
            'u_storm' : u_storm.magnitude if u_storm else None,
            'v_storm' : v_storm.magnitude if v_storm else None,
            'Critical_Angle': critical_angle.magnitude if critical_angle else None,
            'Total_Helicity': total_helicity.magnitude if total_helicity else None,
            })
      except ValueError as e:
        print(f"Error processing {file_name}, sheet {sheet_name}: {e}")
        # Optionally, you can add debugging statements here to inspect the data
        # that caused the error, e.g., print(pressure, temperature, dewpoint)
        continue  # Skip to the next sheet or file

Processing file: /content/drive/MyDrive/Skripsi/WIII012019.xlsx
Processing sheet: 2019-01-01 00Z from file: /content/drive/MyDrive/Skripsi/WIII012019.xlsx
Processing sheet: 2019-01-01 12Z from file: /content/drive/MyDrive/Skripsi/WIII012019.xlsx
Processing sheet: 2019-01-02 00Z from file: /content/drive/MyDrive/Skripsi/WIII012019.xlsx
Processing sheet: 2019-01-02 12Z from file: /content/drive/MyDrive/Skripsi/WIII012019.xlsx
Processing sheet: 2019-01-03 00Z from file: /content/drive/MyDrive/Skripsi/WIII012019.xlsx
Processing sheet: 2019-01-03 12Z from file: /content/drive/MyDrive/Skripsi/WIII012019.xlsx
Processing sheet: 2019-01-04 00Z from file: /content/drive/MyDrive/Skripsi/WIII012019.xlsx
Processing sheet: 2019-01-04 12Z from file: /content/drive/MyDrive/Skripsi/WIII012019.xlsx
Processing sheet: 2019-01-05 00Z from file: /content/drive/MyDrive/Skripsi/WIII012019.xlsx
Processing sheet: 2019-01-05 12Z from file: /content/drive/MyDrive/Skripsi/WIII012019.xlsx
Processing sheet: 2019-01-

ValueError: zero-size array to reduction operation minimum which has no identity

In [ ]:
# Debugging tiap sheet
file_name = excel_files[0]
sheet_name =
sheets_dict = pd.read_excel(file_name, engine="openpyxl", sheet_name=None)
print(f"Processing sheet: {sheet_name} from file: {file_name}")
if {'pressure', 'temperature', 'dewpoint', 'speed', 'direction', 'height'}.issubset(df.columns):
  df = df.dropna(subset=('temperature', 'dewpoint', 'direction', 'speed'), how='all').reset_index(drop=True)
  # Extract data and convert to appropriate units
  pressure = df['pressure'].values * units.hPa
  temperature = df['temperature'].values * units.degC
  dewpoint = df['dewpoint'].values * units.degC
  speed = df['speed'].values * units.knots
  direction = df['direction'].values * units.degrees
  height = df['height'].values * units.meter
  relative_humidity = mpcalc.relative_humidity_from_dewpoint(temperature, dewpoint)
  mixing_ratio = mpcalc.mixing_ratio_from_relative_humidity(pressure, temperature, relative_humidity)

  # Calculate parcel profile
  parcel_prof = mpcalc.parcel_profile(pressure, temperature[0], dewpoint[0])

  # LCL, LFC, CCL and
  lcl_pressure, lcl_temperature = mpcalc.lcl(pressure[0], temperature[0], dewpoint[0])
  lfc_pressure, lfc_temperature = mpcalc.lfc(pressure, temperature, dewpoint)
  el_pressure, el_temperature = mpcalc.el(pressure, temperature, dewpoint, parcel_prof)

  # Mean-layer parcel characteristics (50-hPa depth)
  ml_t, ml_td = mpcalc.mixed_layer(pressure, temperature, dewpoint, depth=100 * units.hPa)
  ml_p, _, _ = mpcalc.mixed_parcel(pressure, temperature, dewpoint, depth=100 * units.hPa)
  mlcape, mlcin = mpcalc.mixed_layer_cape_cin(pressure, temperature, parcel_prof, depth=100 * units.hPa)

  # Most unstable parcel characteristics (50-hPa depth)
  mu_p, mu_t, mu_td, _ = mpcalc.most_unstable_parcel(pressure, temperature, dewpoint, depth=50 * units.hPa)
  mucape, mucin = mpcalc.most_unstable_cape_cin(pressure, temperature, dewpoint, depth=50 * units.hPa)

  # Stability indices
  parcel_p, parcel_t, parcel_td = mpcalc.mixed_parcel(pressure, temperature, dewpoint, height=height, depth=500 * units.meter)
  above = height > 500 * units.m
  press = np.concatenate([[parcel_p], pressure[above]])
  temp = np.concatenate([[parcel_t], temperature[above]])
  dewpt = np.concatenate([[parcel_td], dewpoint[above]])
  mixed_prof = mpcalc.parcel_profile(press, parcel_t, parcel_td)
  lift_index = mpcalc.lifted_index(press, temp, mixed_prof)
  # CAPE and CIN
  p, t, d, prof = mpcalc.parcel_profile_with_lcl(press, temp, dewpt)
  cape, cin = mpcalc.cape_cin(p, t, d, prof)

  k = mpcalc.k_index(pressure, temperature, dewpoint)
  tti = mpcalc.total_totals_index(pressure, temperature, dewpoint)
  cti = mpcalc.cross_totals(pressure, temperature, dewpoint)
  vti = mpcalc.vertical_totals(pressure, temperature)
  si = mpcalc.showalter_index(pressure, temperature, dewpoint)
  sweat = mpcalc.sweat_index(pressure, temperature, dewpoint, speed, direction)
  gdi = mpcalc.galvez_davison_index(pressure, temperature, mixing_ratio, pressure[0])

  # Storm motion and critical angle
  u, v = mpcalc.wind_components(speed, direction)
  (u_storm, v_storm), *_ = mpcalc.bunkers_storm_motion(pressure, u, v, height)
  critical_angle = mpcalc.critical_angle(pressure, u, v, height, u_storm, v_storm)

  # Compute SRH, given a motion vector toward the NE at 9.9 m/s
  *_, total_helicity = mpcalc.storm_relative_helicity(height, u, v, depth=1 * units.km, storm_u=u_storm, storm_v=v_storm)

  # Append results to the list
  stability_indices.append({
      'sheet': datetime.strptime(sheet_name, '%Y-%m-%d %HZ'),
      'LCL_Pressure': lcl_pressure.magnitude,
      'LFC_Pressure': lfc_pressure.magnitude,
      'EL_Pressure': el_pressure.magnitude,
      'CAPE': cape.magnitude,
      'CIN': cin.magnitude,
      'lifted_index': lift_index.magnitude,
      'K_Index': k.magnitude,
      'Total_Totals_Index': tti.magnitude,
      'Cross_Totals_Index': cti.magnitude,
      'Vertical_Totals_Index': vti.magnitude,
      'Showalter_Index': si.magnitude[0],
      'SWEAT_Index': sweat.magnitude[0],
      'Galvez_Davis_Index': gdi.magnitude,
      'Mean_Layer_Temperature': ml_t.magnitude,
      'Mean_Layer_Dewpoint': ml_td.magnitude,
      'Mean_Layer_Pressure': ml_p.magnitude,
      'ML_CAPE': mlcape.magnitude,
      'ML_CIN': mlcin.magnitude,
      'Most_Unstable_Temperature': mu_t.magnitude,
      'Most_Unstable_Dewpoint': mu_td.magnitude,
      'Most_Unstable_Pressure': mu_p.magnitude,
      'MU_CAPE': mucape.magnitude,
      'MU_CIN': mucin.magnitude,
      #'Mean_Parcel_Pressure': mean_parcel_pressure.magnitude,
      #'MU_Parcel_Pressure': mu_parcel_pressure.magnitude,
      #'Bunkers_Storm_Motion': storm_motion,
      'u_storm' : u_storm.magnitude if u_storm else None,
      'v_storm' : v_storm.magnitude if v_storm else None,
      'Critical_Angle': critical_angle.magnitude if critical_angle else None,
      'Total_Helicity': total_helicity.magnitude if total_helicity else None,
      })

Processing file: /content/drive/MyDrive/Skripsi/Data/Banjarmasin/WRBB012019.xlsx
Processing sheet: 2019-01-01 00Z from file: /content/drive/MyDrive/Skripsi/Data/Banjarmasin/WRBB012019.xlsx


ValueError: shape mismatch: objects cannot be broadcast to a single shape.  Mismatch is between arg 0 with shape (92,) and arg 2 with shape (96,).

In [ ]:
# Looping through each file
for file_name in excel_files:
  print(f"Processing file: {file_name}")

  # Reading multiple sheets from an Excel file
  sheets_dict = pd.read_excel(file_name, engine="openpyxl", sheet_name=None)

    # Accessing individual sheets and displaying their contents
  for sheet_name, df in sheets_dict.items():
    print(f"Processing sheet: {sheet_name} from file: {file_name}")
    if {'pressure', 'temperature', 'dewpoint', 'speed', 'direction', 'height'}.issubset(df.columns):
      df = df.dropna(subset=('temperature', 'dewpoint', 'direction', 'speed'), how='all').reset_index(drop=True)
      try:
        # Extract data and convert to appropriate units
        pressure = df['pressure'].values * units.hPa
        temperature = df['temperature'].values * units.degC
        dewpoint = df['dewpoint'].values * units.degC
        speed = df['speed'].values * units.knots
        direction = df['direction'].values * units.degrees
        height = df['height'].values * units.meter
        relative_humidity = mpcalc.relative_humidity_from_dewpoint(temperature, dewpoint)
        mixing_ratio = mpcalc.mixing_ratio_from_relative_humidity(pressure, temperature, relative_humidity)

        # Calculate parcel profile
        parcel_prof = mpcalc.parcel_profile(pressure, temperature[0], dewpoint[0])

        # LCL, LFC, CCL and
        lcl_pressure, lcl_temperature = mpcalc.lcl(pressure[0], temperature[0], dewpoint[0])
        lfc_pressure, lfc_temperature = mpcalc.lfc(pressure, temperature, dewpoint)
        el_pressure, el_temperature = mpcalc.el(pressure, temperature, dewpoint)

        # CAPE and CIN
        p, t, d, prof = mpcalc.parcel_profile_with_lcl()
        cape, cin = mpcalc.cape_cin(pressure, temperature, dewpoint, parcel_prof)

        # Mean-layer parcel characteristics
        ml_t, ml_td = mpcalc.mixed_layer(pressure, temperature, dewpoint, depth=50 * units.hPa)
        ml_p, _, _ = mpcalc.mixed_parcel(pressure, temperature, dewpoint, depth=50 * units.hPa)
        mlcape, mlcin = mpcalc.mixed_layer_cape_cin(pressure, temperature, parcel_prof, depth=50 * units.hPa)

        # Most unstable parcel characteristics
        mu_p, mu_t, mu_td, _ = mpcalc.most_unstable_parcel(pressure, temperature, dewpoint, depth=50 * units.hPa)
        mucape, mucin = mpcalc.most_unstable_cape_cin(pressure, temperature, dewpoint, depth=550 * units.hPa)

        # Stability indices
        lift_index = mpcalc.lifted_index(pressure, temperature, parcel_prof)
        k = mpcalc.k_index(pressure, temperature, dewpoint)
        tti = mpcalc.total_totals_index(pressure, temperature, dewpoint)
        cti = mpcalc.cross_totals(pressure, temperature, dewpoint)
        vti = mpcalc.vertical_totals(pressure, temperature)
        si = mpcalc.showalter_index(pressure, temperature, dewpoint)
        sweat = mpcalc.sweat_index(pressure, temperature, dewpoint, speed, direction)
        gdi = mpcalc.galvez_davison_index(pressure, temperature, mixing_ratio, pressure[0])

        # Storm motion and critical angle
        u, v = mpcalc.wind_components(speed, direction)
        (u_storm, v_storm), *_ = mpcalc.bunkers_storm_motion(pressure, u, v, height)
        critical_angle = mpcalc.critical_angle(pressure, u, v, height, u_storm, v_storm)

        # Compute SRH, given a motion vector toward the NE at 9.9 m/s
        depth = min(height.max() - height.min(), 1 * units.km)
        *_, total_helicity = mpcalc.storm_relative_helicity(height, u, v, depth=depth, storm_u=u_storm, storm_v=v_storm)

        # Append results to the list
        stability_indices.append({
            'sheet': sheet_name,
            'LCL_Pressure': lcl_pressure.magnitude,
            'LCL_Temperature': lcl_temperature.magnitude,
            'LFC_Pressure': lfc_pressure.magnitude if lfc_pressure else None,
            'LFC_Temperature': lfc_temperature.magnitude if lfc_temperature else None,
            'EL_Temperature': el_temperature.magnitude if el_temperature else None,
            'EL_Pressure': el_pressure.magnitude if el_pressure else None,
            'CCL_Pressure': ccl_p.magnitude if ccl_p else None,
            'CAPE': cape.magnitude,
            'CIN': cin.magnitude,
            'lifted_index': lift_index.magnitude,
            'K_Index': k.magnitude,
            'Total_Totals_Index': tti.magnitude,
            'Cross_Totals_Index': cti.magnitude,
            'Vertical_Totals_Index': vti.magnitude,
            'Showalter_Index': si.magnitude,
            'SWEAT_Index': sweat.magnitude,
            'Galvez_Davis_Index': gdi.magnitude,
            'Mean_Layer_Temperature': ml_t.magnitude,
            'Mean_Layer_Dewpoint': ml_td.magnitude,
            'Mean_Layer_Pressure': ml_p.magnitude,
            'ML_CAPE': mlcape.magnitude,
            'ML_CIN': mlcin.magnitude,
            'Most_Unstable_Temperature': mu_t.magnitude,
            'Most_Unstable_Dewpoint': mu_td.magnitude,
            'Most_Unstable_Pressure': mu_p.magnitude,
            'MU_CAPE': mucape.magnitude,
            'MU_CIN': mucin.magnitude,
            #'Mean_Parcel_Pressure': mean_parcel_pressure.magnitude,
            #'MU_Parcel_Pressure': mu_parcel_pressure.magnitude,
            #'Bunkers_Storm_Motion': storm_motion,
            'u_storm' : u_storm.magnitude if u_storm else None,
            'v_storm' : v_storm.magnitude if v_storm else None,
            'Critical_Angle': critical_angle.magnitude if critical_angle else None,
            'Total_Helicity': total_helicity.magnitude if total_helicity else None,
           })
      except ValueError as e:
        print(f"Error processing {file_name}, sheet {sheet_name}: {e}")
        # Optionally, you can add debugging statements here to inspect the data
        # that caused the error, e.g., print(pressure, temperature, dewpoint)
        continue  # Skip to the next sheet or file

In [ ]:
date_str = '02/28/2023 02:30 PM'
date_format = '%m/%d/%Y %I:%M %p'

date_obj = datetime.strptime(date_str, date_format)

NameError: name 'data_obj' is not defined

In [ ]:
def effective_layer(p, t, td, h, height_layer=False):
    """A function that determines the effective inflow layer for a convective sounding.

    Uses the default values of Thompason et al. (2004) for CAPE (100 J/kg) and CIN (-250 J/kg).

    Input:
      - p: sounding pressure with units
      - T: sounding temperature with units
      - Td: sounding dewpoint temperature with units
      - h: sounding heights with units

    Returns:
      - pbot/hbot, ptop/htop: pressure/height of the bottom level,
                              pressure/height of the top level
    """
    from metpy.calc import cape_cin, parcel_profile
    from metpy.units import units

    pbot = None

    for i in range(p.shape[0]):
        prof = parcel_profile(p[i:], t[i], td[i])
        sbcape, sbcin = cape_cin(p[i:], t[i:], td[i:], prof)
        if sbcape >= 100 * units('J/kg') and sbcin > -250 * units('J/kg'):
            pbot = p[i]
            hbot = h[i]
            bot_idx = i
            break
    if not pbot:
        return None, None

    for i in range(bot_idx + 1, p.shape[0]):
        prof = parcel_profile(p[i:], t[i], td[i])
        sbcape, sbcin = cape_cin(p[i:], t[i:], td[i:], prof)
        if sbcape < 100 * units('J/kg') or sbcin < -250 * units('J/kg'):
            ptop = p[i]
            htop = h[i]
            break

    if height_layer:
        return hbot, htop
    else:
        return pbot, ptop

In [ ]:

sheets_dict = pd.read_excel('/content/drive/MyDrive/Skripsi/WIII082019.xlsx', usecols=[2, 3, 4, 5, 7, 8], engine="openpyxl", sheet_name=None)
for sheet_name, df in sheets_dict.items():
  print(f"Sheet '{sheet_name}':\n{df}\n")
  print("\n")  # Print an extra newline for clarity

Sheet '2019-08-01 00Z':
    pressure  height  temperature  dewpoint  direction  speed
0     1012.0       8         22.8      19.4      170.0    3.0
1     1011.0      17         23.6      19.1      172.0    3.0
2     1000.0     115         23.6      18.7      190.0    8.0
3      972.0     363         23.2      20.0      200.0    5.0
4      946.0     598         21.3      18.3      210.0    3.0
..       ...     ...          ...       ...        ...    ...
72      20.0   26380        -57.7       NaN      265.0   22.0
73      19.0   26703        -57.7       NaN      255.0   19.0
74      18.0   27044        -57.7       NaN      140.0    5.0
75      17.0   27405        -57.7       NaN      105.0   20.0
76      16.6   27555        -57.7       NaN        NaN    NaN

[77 rows x 6 columns]



Sheet '2019-08-01 12Z':
    pressure  height  temperature  dewpoint  direction  speed
0     1010.0       8         27.8      20.8      350.0    5.0
1     1000.0      99         27.6      19.6       10.0    

In [ ]:
stability_indices = []
# Process each sheet in the current file
for sheet_name, df in sheets_dict.items():

  # Ensure required columns (temperature, dewpoint, pressure) are present
  if {'pressure', 'temperature', 'dewpoint', 'speed', 'direction', 'height'}.issubset(df.columns):
    df = df.dropna(subset=('temperature', 'dewpoint', 'direction', 'speed'), how='all').reset_index(drop=True)

    try:
      # Extract data and convert to appropriate units
      pressure = df['pressure'].values * units.hPa
      temperature = df['temperature'].values * units.degC
      dewpoint = df['dewpoint'].values * units.degC
      speed = df['speed'].values * units.knots
      direction = df['direction'].values * units.degrees
      height = df['height'].values * units.meter
      relative_humidity = mpcalc.relative_humidity_from_dewpoint(temperature, dewpoint)
      mixing_ratio = mpcalc.mixing_ratio_from_relative_humidity(pressure, temperature, relative_humidity)

      # Calculate parcel profile
      parcel_prof = mpcalc.parcel_profile(pressure, temperature[0], dewpoint[0])

      # LCL, LFC, CCL and
      lcl_pressure, lcl_temperature = mpcalc.lcl(pressure[0], temperature[0], dewpoint[0])
      lfc_pressure, lfc_temperature = mpcalc.lfc(pressure, temperature, dewpoint)
      el_pressure, el_temperature = mpcalc.el(pressure, temperature, dewpoint)
      # CAPE and CIN
      cape, cin = mpcalc.cape_cin(pressure, temperature, dewpoint, parcel_prof)
      # Mean-layer parcel characteristics (50-hPa depth)
      ml_t, ml_td = mpcalc.mixed_layer(pressure, temperature, dewpoint, depth=50 * units.hPa)
      ml_p, _, _ = mpcalc.mixed_parcel(pressure, temperature, dewpoint, depth=50 * units.hPa)
      mlcape, mlcin = mpcalc.mixed_layer_cape_cin(pressure, temperature, parcel_prof, depth=50 * units.hPa)
      # Most unstable parcel characteristics (50-hPa depth)
      mu_p, mu_t, mu_td, _ = mpcalc.most_unstable_parcel(pressure, temperature, dewpoint, depth=50 * units.hPa)
      mucape, mucin = mpcalc.most_unstable_cape_cin(pressure, temperature, dewpoint, depth=50 * units.hPa)
      # Stability indices
      k = mpcalc.k_index(pressure, temperature, dewpoint)
      tti = mpcalc.total_totals_index(pressure, temperature, dewpoint)
      cti = mpcalc.cross_totals(pressure, temperature, dewpoint)
      vti = mpcalc.vertical_totals(pressure, temperature)
      si = mpcalc.showalter_index(pressure, temperature, dewpoint)
      sweat = mpcalc.sweat_index(pressure, temperature, dewpoint, speed, direction)
      gdi = mpcalc.galvez_davison_index(pressure, temperature, mixing_ratio, pressure[0])
      # Storm motion and critical angle
      u, v = mpcalc.wind_components(speed, direction)
      (u_storm, v_storm), *_ = mpcalc.bunkers_storm_motion(pressure, u, v, height)
      critical_angle = mpcalc.critical_angle(pressure, u, v, height, u_storm, v_storm)
      # Compute SRH, given a motion vector toward the NE at 9.9 m/s
      *_, total_helicity = mpcalc.storm_relative_helicity(height, u, v, depth=1 * units.km, storm_u=u_storm, storm_v=v_storm)
      # Append results to the list
      stability_indices.append({
          'sheet': sheet_name,
          'LCL_Pressure': lcl_pressure.magnitude,
          'LFC_Pressure': lfc_pressure.magnitude if lfc_pressure else None,
          'EL_Pressure': el_pressure.magnitude if el_pressure else None,
          'CAPE': cape.magnitude,
          'CIN': cin.magnitude,
          'K_Index': k.magnitude,
          'Total_Totals_Index': tti.magnitude,
          'Cross_Totals_Index': cti.magnitude,
          'Vertical_Totals_Index': vti.magnitude,
          'Showalter_Index': si.magnitude[0],
          'SWEAT_Index': sweat.magnitude[0],
          'Galvez_Davis_Index': gdi.magnitude,
          'Mean_Layer_Temperature': ml_t.magnitude,
          'Mean_Layer_Dewpoint': ml_td.magnitude,
          'Mean_Layer_Pressure': ml_p.magnitude,
          'ML_CAPE': mlcape.magnitude,
          'ML_CIN': mlcin.magnitude,
          'Most_Unstable_Temperature': mu_t.magnitude,
          'Most_Unstable_Dewpoint': mu_td.magnitude,
          'Most_Unstable_Pressure': mu_p.magnitude,
          'MU_CAPE': mucape.magnitude,
          'MU_CIN': mucin.magnitude,
          #'Mean_Parcel_Pressure': mean_parcel_pressure.magnitude,
          #'MU_Parcel_Pressure': mu_parcel_pressure.magnitude,
          #'Bunkers_Storm_Motion': storm_motion,
          'u_storm' : u_storm.magnitude if u_storm else None,
          'v_storm' : v_storm.magnitude if v_storm else None,
          'Critical_Angle': critical_angle.magnitude if critical_angle else None,
          'Total_Helicity': total_helicity.magnitude if total_helicity else None,
          })
    except ValueError as e:
      print(f"Error processing {file_name}, sheet {sheet_name}: {e}")
      # Optionally, you can add debugging statements here to inspect the data
      # that caused the error, e.g., print(pressure, temperature, dewpoint)
      continue  # Skip to the next sheet or file

Error processing /content/drive/MyDrive/Skripsi/WIII122019.xlsx, sheet 2019-12-01 12Z: zero-size array to reduction operation minimum which has no identity
Error processing /content/drive/MyDrive/Skripsi/WIII122019.xlsx, sheet 2019-12-02 12Z: zero-size array to reduction operation minimum which has no identity
Error processing /content/drive/MyDrive/Skripsi/WIII122019.xlsx, sheet 2019-12-06 00Z: zero-size array to reduction operation minimum which has no identity
Error processing /content/drive/MyDrive/Skripsi/WIII122019.xlsx, sheet 2019-12-06 12Z: zero-size array to reduction operation minimum which has no identity


<ipython-input-30-d1bf98421db8>:37: UserWarning: Interpolation point out of data bounds encountered
  k = mpcalc.k_index(pressure, temperature, dewpoint)
<ipython-input-30-d1bf98421db8>:38: UserWarning: Interpolation point out of data bounds encountered
  tti = mpcalc.total_totals_index(pressure, temperature, dewpoint)
<ipython-input-30-d1bf98421db8>:39: UserWarning: Interpolation point out of data bounds encountered
  cti = mpcalc.cross_totals(pressure, temperature, dewpoint)
<ipython-input-30-d1bf98421db8>:40: UserWarning: Interpolation point out of data bounds encountered
  vti = mpcalc.vertical_totals(pressure, temperature)
<ipython-input-30-d1bf98421db8>:41: UserWarning: Interpolation point out of data bounds encountered
  si = mpcalc.showalter_index(pressure, temperature, dewpoint)
<ipython-input-30-d1bf98421db8>:42: UserWarning: Interpolation point out of data bounds encountered
  sweat = mpcalc.sweat_index(pressure, temperature, dewpoint, speed, direction)
<ipython-input-30-d1b

Error processing /content/drive/MyDrive/Skripsi/WIII122019.xlsx, sheet 2019-12-14 12Z: Specified bound is outside height range.


KeyboardInterrupt: 

In [ ]:
import pandas as pd
import glob
from metpy.calc import relative_humidity_from_dewpoint
from metpy.units import units
import os

# Mount Google Drive if the files are stored there (uncomment if needed)
# from google.colab import drive
# drive.mount('/content/drive')

# Specify the directory containing the Excel files
# Replace this with the correct directory path in your Colab environment
directory_path = "/content/drive/MyDrive/Skripsi"  # Adjust as needed

# Check if the directory exists
if not os.path.exists(directory_path):
    print(f"Directory {directory_path} does not exist. Please check the path.")
else:
    # Use glob to find all Excel files in the directory
    file_pattern = "*.xlsx"  # Pattern to match Excel files
    excel_files = glob.glob(f"{directory_path}/{file_pattern}")

    if not excel_files:
        print(f"No Excel files found in {directory_path}.")
    else:
        # Initialize a list to store the processed DataFrames for all sheets in all files
        all_processed_dataframes = []

        # Iterate over each Excel file
        for file_path in excel_files:
            print(f"Processing file: {file_path}")
            # Load all sheets into a dictionary of DataFrames
            try:
                sheets_dict = pd.read_excel(file_path, sheet_name=None)

                # Process each sheet
                for sheet_name, df in sheets_dict.items():
                    print(f"Processing sheet: {sheet_name} from file: {file_path}")
                    all_processed_dataframes.append((file_path, sheet_name, df))

            except Exception as e:
                print(f"Error reading {file_path}: {e}")

        print("All files processed.")

Processing file: /content/drive/MyDrive/Skripsi/WIII012019.xlsx
Processing sheet: 2019-01-01 00Z from file: /content/drive/MyDrive/Skripsi/WIII012019.xlsx
Processing sheet: 2019-01-01 12Z from file: /content/drive/MyDrive/Skripsi/WIII012019.xlsx
Processing sheet: 2019-01-02 00Z from file: /content/drive/MyDrive/Skripsi/WIII012019.xlsx
Processing sheet: 2019-01-02 12Z from file: /content/drive/MyDrive/Skripsi/WIII012019.xlsx
Processing sheet: 2019-01-03 00Z from file: /content/drive/MyDrive/Skripsi/WIII012019.xlsx
Processing sheet: 2019-01-03 12Z from file: /content/drive/MyDrive/Skripsi/WIII012019.xlsx
Processing sheet: 2019-01-04 00Z from file: /content/drive/MyDrive/Skripsi/WIII012019.xlsx
Processing sheet: 2019-01-04 12Z from file: /content/drive/MyDrive/Skripsi/WIII012019.xlsx
Processing sheet: 2019-01-05 00Z from file: /content/drive/MyDrive/Skripsi/WIII012019.xlsx
Processing sheet: 2019-01-05 12Z from file: /content/drive/MyDrive/Skripsi/WIII012019.xlsx
Processing sheet: 2019-01-

In [ ]:
directory_path = "/content/drive/MyDrive/Skripsi"
if not os.path.exists(directory_path):
    print(f"Directory {directory_path} does not exist. Please check the path.")
else:
    file_pattern = "*.xlsx"  # Pattern to match Excel files
    excel_files = glob.glob(f"{directory_path}/{file_pattern}")
    if not excel_files:
        print(f"No Excel files found in {directory_path}.")
    else:
      file = pd.DataFrame()
      for file_path in excel_files:
        print(f"Processing file: {file_path}")
        try:
          sheets_dict = pd.read_excel(file_path, sheet_name=None)
          file.append(sheets_dict)
        except Exception as e:
          print(f"Error reading {file_path}: {e}")
      sheets_dict = pd.concat(file, axis=0, ignore_index=True)
      print("All files processed.")

Processing file: /content/drive/MyDrive/Skripsi/WIII012019.xlsx
Error reading /content/drive/MyDrive/Skripsi/WIII012019.xlsx: 'DataFrame' object has no attribute 'append'
Processing file: /content/drive/MyDrive/Skripsi/WIII022019.xlsx
Error reading /content/drive/MyDrive/Skripsi/WIII022019.xlsx: 'DataFrame' object has no attribute 'append'
Processing file: /content/drive/MyDrive/Skripsi/WIII032019.xlsx
Error reading /content/drive/MyDrive/Skripsi/WIII032019.xlsx: 'DataFrame' object has no attribute 'append'
Processing file: /content/drive/MyDrive/Skripsi/WIII042019.xlsx
Error reading /content/drive/MyDrive/Skripsi/WIII042019.xlsx: 'DataFrame' object has no attribute 'append'
Processing file: /content/drive/MyDrive/Skripsi/WIII052019.xlsx


KeyboardInterrupt: 

In [ ]:
excel_files

['/content/drive/MyDrive/Skripsi/WIII012019.xlsx',
 '/content/drive/MyDrive/Skripsi/WIII022019.xlsx',
 '/content/drive/MyDrive/Skripsi/WIII032019.xlsx',
 '/content/drive/MyDrive/Skripsi/WIII042019.xlsx',
 '/content/drive/MyDrive/Skripsi/WIII052019.xlsx',
 '/content/drive/MyDrive/Skripsi/WIII062019.xlsx',
 '/content/drive/MyDrive/Skripsi/WIII072019.xlsx',
 '/content/drive/MyDrive/Skripsi/WIII082019.xlsx',
 '/content/drive/MyDrive/Skripsi/WIII092019.xlsx',
 '/content/drive/MyDrive/Skripsi/WIII102019.xlsx',
 '/content/drive/MyDrive/Skripsi/WIII112019.xlsx',
 '/content/drive/MyDrive/Skripsi/WIII122019.xlsx']

In [ ]:
# Iterate over each Excel file
for file_path in excel_files:
    # Load all sheets into a dictionary of DataFrames
    sheets_dict = pd.read_excel(file_path, sheet_name=None)

    # Process each sheet in the current file
    for sheet_name, df in sheets_dict.items():

        # Check for necessary columns in the current sheet
        if 'temperature' in df.columns and 'pressure' in df.columns and 'dewpoint' in df.columns:

            # Convert to appropriate units
            temperature = df['temperature'].values * units.degC
            dewpoint = df['dewpoint'].values * units.degC
            pressure = df['pressure'].values * units.hPa

            # Calculate relative humidity
            relative_humidity = mpcalc.relative_humidity_from_dewpoint(temperature, dewpoint) * units.dimensionless
            df['relative_humidity'] = relative_humidity.magnitude

            # Calculate mixing ratiorelative_humidity, temperature, pressure
            mixing_ratio = mpcalc.mixing_ratio_from_relative_humidity(pressure, temperature, relative_humidity)
            df['mixing_ratio'] = mixing_ratio.magnitude

            # Calculate potential temperature
            potential_temp = mpcalc.potential_temperature(pressure, temperature)
            df['potential_temperature'] = potential_temp.magnitude

            # Calculate equivalent potential temperature
            equiv_potential_temp = mpcalc.equivalent_potential_temperature(pressure, temperature, dewpoint)
            df['equivalent_potential_temperature'] = equiv_potential_temp.magnitude

            # Calculate virtual potential temperature
            virtual_potential_temp = mpcalc.virtual_potential_temperature(pressure, temperature, mixing_ratio)
            df['virtual_potential_temperature'] = virtual_potential_temp.magnitude

            # Append processed DataFrame with the calculated columns to the list
            all_processed_dataframes.append((file_path, sheet_name, df))

In [ ]:
for sheet_name, df in sheets_dict.items():
  # Ensure required columns (temperature, dewpoint, pressure) are present
  if {'pressure', 'temperature', 'dewpoint', 'speed', 'direction', 'height'}.issubset(df.columns):
    df = df.dropna(subset=('temperature', 'dewpoint', 'direction', 'speed'), how='all').reset_index(drop=True)